In [6]:
from occultation_detector.difractions import *

In [7]:
import matplotlib
import matplotlib.pyplot as p

In [8]:
#Parametros basicos para el calculo
M=2**11 # Tamano de la malla en [px] 2048
lamb=600e-9 # Long de onda en [m]


#Parametros de la observacion (conocidos a Priori)
vE=29800 # velocidad de traslacion de la tierra  en m/s
vr=5000 #velocidad del cuerpo Pos si va en contra de la direccion de la tierra
ang=30 #angulo desde oposicion para calcular velocidad tangencial del objeto
fps=20 #frames por segundo
mV=15 # Magnitud aparente de la estrella # TODO: Magnitud 15, 12
nEst=1 #Seleccion de tipo espectral de estrella # TODO: 1,30 
nLamb=10 # Num de longitudes de onda a considerar para el calculo espectral spectra()
snr = SNR_TAOS2(mV) #Calcular senal a ruido segun TAOS-2

# TODO: Estrella tipo A0
#A0=1;A1=2;A2=3;A3=4;A4=5;A5=6;A7=7;F0=8;F2=9;F3=10;F5=11;F6=12;F7=13;F8=14
#G0=15;G1=16;G2=17;G5=18;G8=19;K0=20;K1=21;K2=22;K3=23;K4=24;K5=25;K7=26;
#M0=27;M1=28;M2=29;M3=30;M4=31;M5=32;M6=33;M7=34;M8=35


#Parametros del evento de ocultacion (desconocidos) incluir d
d=3000 # Diam del objeto en [m]
ua=45 #Distancia del objeto en unidades astronómicas
D=calc_plano(d,lamb,ua) #Tamano del plano total en [m]
toffset=0 #en pixeles TODO
T=0 # direccion de lectura en grados
b=0 # Parametro de impacto en metros

In [24]:
D

142112.63138792414

In [28]:
from collections import OrderedDict

def simulate_circle_lightcurve(diameter, astronomical_unit, time_offset, angle_in_degrees, baseline):
    total_plane_size = calc_plano(diameter, lamb, astronomical_unit)  # Size of the total plane in meters
    circular_pupil = pupilCO(M, total_plane_size, diameter)  # Circular pupil object

    # CALCULATE PATTERN WITH SPECTRAL CONTRIBUTION
    object_distance = 1.496e11 * astronomical_unit  # Distance of the object in meters
    monochromatic_diffraction_pattern = fresnel(circular_pupil, M, total_plane_size, object_distance, lamb)
    chromatic_diffraction_pattern = spectra(circular_pupil, M, total_plane_size, object_distance, nEst, nLamb)

    # CALCULATE PATTERN FOR EXTENDED SOURCE
    star_type, star_radius = calc_rstar(mV, nEst, astronomical_unit)  # Calculate the star's radius and type using stars.dat
    extended_source_pattern = promedio_PD(chromatic_diffraction_pattern, star_radius, total_plane_size, M, diameter)

    # ADD POISSON NOISE
    noisy_diffraction_pattern = add_ruido(extended_source_pattern, mV)

    # EXTRACT DIFFRACTION PROFILE
    # `angle_in_degrees` represents angular positions; `baseline` represents baseline distance in meters
    profile_without_noise_x, profile_without_noise_y = extraer_perfil(extended_source_pattern, M, total_plane_size, angle_in_degrees, baseline)
    profile_with_noise_x, profile_with_noise_y = extraer_perfil(noisy_diffraction_pattern, M, total_plane_size, angle_in_degrees, baseline)

    # SAMPLE BASED ON PREDEFINED PARAMETERS
    sampled_x1, sampled_y1, sampled_x2, sampled_y2 = muestreos(
        profile_without_noise_y, total_plane_size, vr, fps,
        toff=time_offset, vE=vE, opangle=0, ua=astronomical_unit
    )

    return {
         "total_plane_size": total_plane_size,
        "object_distance": object_distance,
        "star_radius": star_radius,
        "star_type": star_type,
        "total_plane_size": total_plane_size,
        "object_distance": object_distance,
        "star_radius": star_radius,
        "star_type": star_type,
        "circular_pupil": circular_pupil,
        "monochromatic_diffraction_pattern": monochromatic_diffraction_pattern,
        "chromatic_diffraction_pattern": chromatic_diffraction_pattern,
        "extended_source_pattern": extended_source_pattern,
        "noisy_diffraction_pattern": noisy_diffraction_pattern,
        "profile_without_noise_x": profile_without_noise_x,
        "profile_without_noise_y": profile_without_noise_y,
        "profile_with_noise_x": profile_with_noise_x,
        "profile_with_noise_y": profile_with_noise_y,
        "sampled_x1": sampled_x1,
        "sampled_y1": sampled_y1,
        "sampled_x2": sampled_x2,
        "sampled_y2": sampled_y2,
    }


In [29]:
x=simulate_circle_lightcurve(d, ua, toffset, T, b)
x

{'total_plane_size': 142112.63138792414,
 'object_distance': 6732000000000.0,
 'star_radius': 53.816693374514756,
 'star_type': 'A0',
 'circular_pupil': array([[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]),
 'monochromatic_diffraction_pattern': array([[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]),
 'chromatic_diffraction_pattern': array([[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]),
 'extended_source_pattern': arr

In [23]:
import h5py
import numpy as np

def save_to_hdf5(data, filename):
    """
    Save the output of simulate_circle_lightcurve to an HDF5 file.
    
    Parameters:
        data (tuple): Output of simulate_circle_lightcurve.
        filename (str): HDF5 file name.
    """
    with h5py.File(filename, 'w') as h5file:
        for key, value in data.items():
            # Save scalars or arrays
            if hasattr(value, '__len__') and not isinstance(value, str):
                h5file.create_dataset(key, data=value)
            else:
                h5file.create_dataset(key, data=value)

# Save to HDF5
save_to_hdf5(x[1], 'testing_simulation.h5')

In [13]:
import h5py
import numpy as np

# Create a new HDF5 file
with h5py.File('testing_simulation.h5', 'w') as file:
    file.create_dataset('object', data=data)
    file.attrs['description'] = 'This is an example file'

NameError: name 'data' is not defined

# Web server

In [3]:
import gradio as gr

def update(d, ua, toffset, T, b):
    plots = [None] * 9
    Lims=20 #lims in km
    response,series = simulate_circle_lightcurve(d,ua,toffset,T,b)
    extent=[-response['D']/2000,response['D']/2000,-response['D']/2000,response['D']/2000]
    plots[0]=p.figure(1)
    p.clf()
    p.imshow(series['O1'],extent=extent)
    p.xlabel('Distance [km]')
    p.ylabel('Distance [km]')
    p.xlim([-2*d/1000,2*d/1000])
    p.ylim([-2*d/1000,2*d/1000])
    p.gray()
    p.title('Circular Object')
    
    plots[1]=p.figure(2)
    p.clf()
    p.imshow(series['I1'],extent=extent)
    p.gray()
    p.xlabel('Distance [km]')
    p.ylabel('Distance [km]')
    p.xlim([-Lims,Lims])
    p.ylim([-Lims,Lims])
    p.title('Diffraction pattern, circular object')

    plots[2] = p.figure(3)
    p.clf()
    p.imshow(series['I1s'],extent=extent)
    p.gray()
    p.xlabel('Distance [km]')
    p.ylabel('Distance [km]')
    p.xlim([-Lims,Lims])
    p.ylim([-Lims,Lims])
    p.title('Chromatic diffraction pattern, circular: '+response['tipo']+', Longs. Onda: '+str(np.round(nLamb)))

    plots[3] = p.figure(4)
    p.clf()
    p.imshow(series['I1n'],extent=extent)
    p.gray()
    p.xlabel('Distance [km]')
    p.ylabel('Distance [km]')
    p.xlim([-Lims,Lims])
    p.ylim([-Lims,Lims])
    p.title('Diffraction pattern, circular, noise mV: '+str(mV)+', SNR: '+str(np.round(snr)))

    plots[4] = p.figure(5)
    p.clf()
    p.plot(series['xc']/1000,series['yc'])
    p.xlim([-Lims,Lims])
    p.xlabel('Distance [km]')
    p.ylabel('Normalized intensity')
    p.title('Circular diffraction profile')

    plots[5] = p.figure(6)
    p.clf()
    p.plot(series['xb']/1000,series['yb'])
    p.xlim([-Lims,Lims])
    p.xlabel('Distance [km]')
    p.ylabel('Normalized intensity')
    p.title('Binary diffraction profile')

    plots[6] = p.figure(7)
    p.clf()
    p.plot(series['x1'],series['y1'])
    p.xlabel('Time [s]')
    p.ylabel('Normalized intensity without noise')
    p.xlim([-1,1])
    
    # dat=pd.DataFrame(series['x1'],series['y1'])
    # dat.to_csv('results/CB_D10_pattern_b%d.csv'% (b/1000))
    # p.savefig('results/CB_D10_pattern_b%d.png'% (b/1000))
    
    plots[7] = p.figure(8)
    p.clf()
    p.plot(series['x2'],series['y2'],'.-')
    p.xlabel('Time [s]')
    p.ylabel('Normalized intensity with noise')

    plots[8] = response
    return tuple(plots)

with gr.Blocks() as demo:
    gr.Markdown(
    """
    # TNO Simulator
    ## TAOS II
    """)
    with gr.Row():
        with gr.Column(scale=1, min_width=600):
            d_input = gr.Number(label="Diámetro del objeto en [m]", value=3000)
            ua_input = gr.Number(label="Distancia del objeto en unidades astronómicas", value=45)
            lamb_input = gr.Number(label="Wavelength", value=0)
            toffset_input = gr.Number(label="Offset en pixeles", value=0)
            T_input = gr.Slider(label="Dirección de lectura en grados", minimum=0, maximum=360, value=0)
            b_input = gr.Number(label="Parámetro de impacto en metros", value=0)
            btn = gr.Button("Run")
        with gr.Column(scale=2, min_width=600):
            outputs = [
                *[gr.Plot(label="Plot") for _ in range(0,8)],
                gr.JSON()
            ]
    btn.click(fn=update, inputs=[d_input, ua_input, toffset_input, T_input, b_input], outputs=outputs)


demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://5f65336cb15032682c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Combinations

In [27]:
import itertools
import pandas as pd
from collections import OrderedDict
import numpy as np

#Parametros basicos para el calculo
M=2**11 # Tamano de la malla en [px] 2048
lamb=600e-9 # Long de onda en [m]

#Parametros de la observacion (conocidos a Priori)
vE=29800 # velocidad de traslacion de la tierra  en m/s
vr=5000 #velocidad del cuerpo Pos si va en contra de la direccion de la tierra
ang=30 #angulo desde oposicion para calcular velocidad tangencial del objeto
fps=20 #frames por segundo
mV=12 # Magnitud aparente de la estrella
nEst=30 #Seleccion de tipo espectral de estrella
nLamb=10 # Num de longitudes de onda a considerar para el calculo espectral spectra()
snr = SNR_TAOS2(mV) #Calcular senal a ruido segun TAOS-2

# Tipo de estrella
#A0=1;A1=2;A2=3;A3=4;A4=5;A5=6;A7=7;F0=8;F2=9;F3=10;F5=11;F6=12;F7=13;F8=14
#G0=15;G1=16;G2=17;G5=18;G8=19;K0=20;K1=21;K2=22;K3=23;K4=24;K5=25;K7=26;
#M0=27;M1=28;M2=29;M3=30;M4=31;M5=32;M6=33;M7=34;M8=35

# grid
set_diameters = np.linspace(1000,10000,1)
set_ua = np.linspace(40,60,1)
set_toffset = [0]
set_T = [0]
set_b_impact_parameter = np.linspace(0,3,1) # 0 a 3, multiplicas por el diametro
n = 20*5*41 # number of combinations

def simulate_circle_lightcurve(d, ua, toffset, T, b):
    D = calc_plano(d, lamb, ua) #Tamano del plano total en [m]
    O1 = pupilCO(M, D, d)  #Objeto 1: circular
    # CALCULAR PATRON CON CONTRIBUCION ESPECTRAL
    z = 1.496e11 * ua #Distancia del objeto en [m]
    I1s = spectra(O1, M, D, z, nEst, nLamb) #Esta funcion calcula el patron cromatico
    #CALCULAR PATRON PARA FUENTE EXTENDIDA
    tipo, R_star = calc_rstar(mV, nEst, ua) #Funcion para calcular el radio y tipo de la estrella usa estrellas.dat
    I1f = promedio_PD(I1s, R_star, D, M, d) #Funcion para calcular contribucion de fuente extendida
    #EXTRAER PERFIL DE DIFRACCION ojo T--> grados y b --> metros
    _, yc = extraer_perfil(I1f, M, D, T, b) #Extraer perfil de difraccion sin ruido
    #MUESTREAR SEGUN PARAMETROS CONOCIDOS DEFINIDOS AL PRINCIPIO
    _, _, x2, y2 = muestreos(yc, D, vr, fps, toff=toffset, vE=vE, opangle=0, ua=ua)  #FUNCION PARA MUESTREAR genera dos tuplas
    
    return (OrderedDict({
        "D": D,
        "z": z,
        "R_star": R_star,
        "tipo": tipo
    }), {
        "x2": x2,
        "y2": y2,
    })

import concurrent.futures
import os 


def f(ordered_pair):
   response, series = simulate_circle_lightcurve(*ordered_pair)
   return [*ordered_pair, *list(response.values()), np.vstack((series['x2'],series['y2']))]


pairs=list(itertools.product(set_diameters,set_ua,set_toffset,set_T,set_b_impact_parameter))[:2]

y=[f(pair) for pair in pairs]
y

[[1000.0,
  40.0,
  0,
  0,
  0.0,
  133985.0737955538,
  5984000000000.0,
  3324.659142674617,
  'M3',
  array([[-2.22653807, -2.17760317, -2.12866827, -2.07973337, -2.03079846,
          -1.98186356, -1.93292866, -1.88399376, -1.83505885, -1.78612395,
          -1.73718905, -1.68825414, -1.63931924, -1.59038434, -1.54144944,
          -1.49251453, -1.44357963, -1.39464473, -1.34570983, -1.29677492,
          -1.24784002, -1.19890512, -1.14997021, -1.10103531, -1.05210041,
          -1.00316551, -0.9542306 , -0.9052957 , -0.8563608 , -0.8074259 ,
          -0.75849099, -0.70955609, -0.66062119, -0.61168628, -0.56275138,
          -0.51381648, -0.46488158, -0.41594667, -0.36701177, -0.31807687,
          -0.26914197, -0.22020706, -0.17127216, -0.12233726, -0.07340235,
          -0.02446745,  0.02446745,  0.07340235,  0.12233726,  0.17127216,
           0.22020706,  0.26914197,  0.31807687,  0.36701177,  0.41594667,
           0.46488158,  0.51381648,  0.56275138,  0.61168628,  0.660621

In [31]:
[pair[0:-1] for pair in y]

[[1000.0,
  40.0,
  0,
  0,
  0.0,
  133985.0737955538,
  5984000000000.0,
  3324.659142674617,
  'M3']]

In [37]:
for index,pair in enumerate(y):
    np.save(f'results/lightcurve-{index}.npy', np.array(np.array(pair[-1])))